In [1]:
# -*- coding: utf-8 -*-
import argparse
import copy
import json
import os
import pickle
import random
import sys
import re
import numpy as np
import openai

from utils import *

GPT_MODEL = "gpt-4-turbo-preview"  # [gpt-4-turbo-preview]

setOpenAi(keyid = 2)

0

### Task: Last_First_Concatenate
- 首先拆分成小任务
- 然后逐个任务串联
- 纯粹的级联结构即可,无需更加复杂的结构

In [5]:

GPT_MODEL = "gpt-4-turbo-preview"  # [gpt-4-turbo-preview]]

problem = """
        "Texture, Mango, Bridge"
        """

decompose = {"role": "system", 
                "content": """
        I will give you a list of words. And you need to decompose the list of words in to sublists of the given list.
        Two examples are as follows:
        
        Q: “machine, learning”
        A: creating sequential sublists of the list “machine, learning”: 
        “machine”
        “machine, learning”

        Q: “machine, learning, artificial”
        A: creating sequential sublists of the list “machine, learning, artificial”: 
        “machine”
        “machine, learning”
        “machine, learning, artificial”
        """}

question_last_word = {"role": "user", "content": f"""Now the list of words is {problem}, please decompose it into smaller lists like the examples. Format: "list" can be solved by: "sublist_0", "sublist_1",...  """}

Q_decompose = [decompose, question_last_word]
decom_lists = askChatGPT(Q_decompose, model=GPT_MODEL, temperature = 1)
print(decom_lists)

#匹配双引号中的内容
pattern = r'"([^"]*)"'

# 查找所有匹配项
matches = re.findall(pattern, decom_lists)

# 提取匹配内容并转换为列表
if matches:
    list_decomposed = matches[1:]  # 忽略第一个匹配项
    print(list_decomposed)
else:
    print("没有找到匹配的内容")

The list "Texture, Mango, Bridge" can be solved by: 
- "Texture"
- "Texture, Mango"
- "Texture, Mango, Bridge"
['Texture', 'Texture, Mango', 'Texture, Mango, Bridge']



然后需要对于每个任务进行解答，然后将解答串联起来


In [6]:
concenate_method = {"role": "system", "content": f"""I need you to solve the list by concatenating the last letter of the words in the sublists. I've broken command down into smaller sublists for you. Please follow the sublists I provided.
The initial command is {list_decomposed}.

Four examples are as follows:

Q: “think, machine”
A: The last letter of “think” is “k”. The last letter of “machine” is “e”. Concatenating “k”, “e” leads to “ke”. So, “think, machine” outputs “ke”.

Q: “think, machine, learning”
A: “think, machine” outputs “ke”. The last letter of “learning” is “g”. Concatenating “ke”, “g” leads to “keg”. So, “think, machine, learning” outputs “keg”.

Q: “transformer, language”
A: The last letter of “transformer” is “r”. The last letter of “language” is “e”. Concatenating: “r”, “e” leads to “re”. So, “transformer, language” outputs “re”.

Q: “transformer, language, vision”
A: “transformer, language” outputs “re”. The last letter of “vision” is “n”. Concatenating: “re”, “n” leads to “ren”. So, “transformer, language, vision” outputs “ren”.

"""}

Q_now = [concenate_method]

for i in range(len(list_decomposed)):
    sublist = list_decomposed[i]
    Q_sub = {"role": "user", "content": f"""Subcommand: {sublist}. Please retrieve and concatenate the last letter."""}
    Q_now.append(Q_sub)
    answer_sub = askChatGPT(Q_now, model=GPT_MODEL, temperature = 1)
    print(answer_sub)
    Q_now.append({"role": "assistant", "content": answer_sub})

Q_final = {"role": "user", "content": """So the final answer is: """}  # 不论subquesition有没有覆盖到最后一问,都统一问一下
Q_now.append(Q_final)
printSeq(Q_now)
answer_final = askChatGPT(Q_now, model=GPT_MODEL, temperature = 1)
print(answer_final)


The last letter of "Texture" is "e". Thus, the output is "e".
The last letter of "Texture" is "e". The last letter of "Mango" is "o". Concatenating "e", "o" leads to "eo". So, the output is "eo".
The output from "Texture, Mango" is "eo". The last letter of "Bridge" is "e". Concatenating "eo", "e" leads to "eoe". So, the output is "eoe".
{'role': 'system', 'content': "I need you to solve the list by concatenating the last letter of the words in the sublists. I've broken command down into smaller sublists for you. Please follow the sublists I provided.\nThe initial command is ['Texture', 'Texture, Mango', 'Texture, Mango, Bridge'].\n\nFour examples are as follows:\n\nQ: “think, machine”\nA: The last letter of “think” is “k”. The last letter of “machine” is “e”. Concatenating “k”, “e” leads to “ke”. So, “think, machine” outputs “ke”.\n\nQ: “think, machine, learning”\nA: “think, machine” outputs “ke”. The last letter of “learning” is “g”. Concatenating “ke”, “g” leads to “keg”. So, “think,